In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

df1 = pd.read_csv('/content/drive/MyDrive/데이터 셋/P220404-01.csv')
print(df1.shape)
print(df1.info())
print(df1.describe())
print(df1.isnull().sum())
print(df1.head())
print(df1['Segmentation'].value_counts())
print(df1['Profession'].unique())

df2 = pd.read_csv('/content/drive/MyDrive/데이터 셋/P220404-02.csv')
print(df2.shape)
print(df2.info())
print(df2.describe())
print(df2.isnull().sum())
print(df2.head())

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler

le = LabelEncoder()
label = ['Profession']
df1[label] = le.fit_transform(df1[label]).reshape(-1,1)
df2[label] = le.fit_transform(df2[label]).reshape(-1,1)

df1 = pd.get_dummies(df1, columns = ['Gender', 'Ever_Married','Graduated','Spending_Score','Var_1'])
df2 = pd.get_dummies(df2, columns = ['Gender', 'Ever_Married','Graduated','Spending_Score','Var_1'])

mms = MinMaxScaler()
numbers = ['Age','Work_Experience','Family_Size']
df1[numbers] = mms.fit_transform(df1[numbers])
df2[numbers] = mms.fit_transform(df2[numbers])

print(df1)
print(df2)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score

df1['Segmentation'] = df1['Segmentation'].astype('category')

x = df1.drop('Segmentation', axis=1)
y = df1['Segmentation']

x_train, x_valid, y_train, y_valid = train_test_split(x, y, test_size=0.2,random_state=42)

md = RandomForestClassifier(n_estimators=50, max_depth=6)
md.fit(x_train.drop('ID', axis=1), y_train)

pred = md.predict(x_valid.drop('ID',axis=1))

cm = confusion_matrix(y_valid, pred, labels=['A','B','C','D'])
print(cm)
print(accuracy_score(y_valid, pred))

print(f1_score(y_valid, pred, average='macro'))

[[169  45  41  60]
 [ 93  72 107  30]
 [ 41  48 203  54]
 [ 89  19  14 248]]
0.5191297824456114
0.4990795083647088


In [ ]:
from sklearn.ensemble import AdaBoostClassifier

md2= AdaBoostClassifier()
md2.fit(x_train.drop('ID',axis=1), y_train)
pred = md.predict(x_valid.drop('ID',axis=1))
print(pred)

cm2 = confusion_matrix(y_valid, pred)
print(cm)
print(accuracy_score(y_valid, pred))
print(f1_score(y_valid, pred, average='macro'))

['D' 'A' 'D' ... 'C' 'C' 'C']
[[137  64  50  64]
 [ 77  97 103  25]
 [ 39  89 175  43]
 [ 85  25  25 235]]
0.4831207801950488
0.4735648677024219


In [ ]:
from lightgbm import LGBMClassifier
md3 = LGBMClassifier()
md3.fit(x_train.drop('ID',axis=1), y_train)
pred = md3.predict(x_valid.drop('ID',axis=1))
print(pred)

cm3 = confusion_matrix(y_valid, pred)
print(cm3)
print(accuracy_score(y_valid, pred))
print(f1_score(y_valid, pred, average='macro'))

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000300 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 132
[LightGBM] [Info] Number of data points in the train set: 5332, number of used features: 20
[LightGBM] [Info] Start training from score -1.410593
[LightGBM] [Info] Start training from score -1.434710
[LightGBM] [Info] Start training from score -1.356000
[LightGBM] [Info] Start training from score -1.346583
['D' 'A' 'D' ... 'C' 'C' 'A']
[[141  64  41  69]
 [ 73  95 106  28]
 [ 34  64 200  48]
 [ 81  15  22 252]]
0.5161290322580645
0.501571554077081


In [ ]:
from sklearn.ensemble import VotingClassifier

md4 = VotingClassifier(estimators= [('rf',RandomForestClassifier(n_estimators=300)), ('ad',AdaBoostClassifier)], \
                      voting = 'soft')
md4.fit(x_train.drop('ID', axis=1),y_train)
pred = md4.fit(x_valid.drop('ID', axis=1),y_valid)

print(accuracy_score(y_valid, pred))
print(f1_score(y_valid, pred, average='macro'))

TypeError: Cannot clone object. You should provide an instance of scikit-learn estimator instead of a class.

In [ ]:
from sklearn.model_selection import GridSearchCV

params = {'n_estimators':[50,100], 'max_depth':[4,6]}
md5 = RandomForestClassifier()
gcv = GridSearchCV(estimator = md5, param_grid = params, cv=3)
gcv.fit(x_train.drop('ID', axis=1),y_train)
print(gcv.best_params_)

{'max_depth': 6, 'n_estimators': 50}


In [ ]:
x_test = df2

md.fit(x.drop('ID',axis=1), y)
pred = md.predict(x_test.drop('ID',axis=1))
print(pred)

result = pd.DataFrame({'ID': df2['ID'], 'pred': pred})
print(result)
result.to_csv("수업번호.csv", index=False)

['B' 'C' 'C' ... 'B' 'C' 'D']
          ID pred
0     458989    B
1     458994    C
2     459000    C
3     459003    C
4     459005    A
...      ...  ...
2149  467950    A
2150  467954    D
2151  467958    B
2152  467961    C
2153  467968    D

[2154 rows x 2 columns]


In [ ]:
import pandas as pd

train = pd.read_csv('/content/drive/MyDrive/데이터 셋/P220504-01.csv')
print(train.shape)
print(train.head())
print(train.info())

print(train.describe())
print(train.isnull().sum())
print(train['year'].value_counts())
print(train['transmission'].value_counts())
print(train['fuelType'].value_counts())


In [ ]:
test = pd.read_csv('/content/drive/MyDrive/데이터 셋/P220504-02.csv')
print(test.shape)
print(test.head())
print(test.info())
print(test.describe())
print(test.isnull().sum())
print(test['fuelType'].value_counts())

(3882, 8)
       model  year transmission  mileage fuelType  tax   mpg  engineSize
0   1 Series  2018    Semi-Auto    14200   Petrol  145  36.7         1.5
1   3 Series  2019    Semi-Auto     2150   Petrol  145  42.2         2.0
2   4 Series  2019    Semi-Auto     6650   Petrol  145  48.7         2.0
3   1 Series  2019    Semi-Auto     2604   Diesel  145  70.6         2.0
4         X3  2016    Semi-Auto    35700   Diesel  145  54.3         2.0
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3882 entries, 0 to 3881
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   model         3882 non-null   object 
 1   year          3882 non-null   int64  
 2   transmission  3882 non-null   object 
 3   mileage       3882 non-null   int64  
 4   fuelType      3882 non-null   object 
 5   tax           3882 non-null   int64  
 6   mpg           3882 non-null   float64
 7   engineSize    3882 non-null   float64
dtypes: float64(

In [ ]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OrdinalEncoder

le = OrdinalEncoder()
label = ['year', 'model']
train[label] = le.fit_transform(train[label])
test[label] = le.fit_transform(test[label])

mms = MinMaxScaler()
number = ['mileage', 'tax', 'mpg', 'engineSize']
train[number] = mms.fit_transform(train[number])
test[number] = mms.fit_transform(test[number])

ohe = OneHotEncoder(sparse=False)
ohe_train = ohe.fit_transform(train[['transmission','fuelType']])
ohe_df_train = pd.DataFrame(ohe_train, columns=ohe.get_feature_names_out())
train_df = pd.concat([ohe_df_train, train.drop(['transmission','fuelType'], axis=1)],axis=1)

ohe_test = ohe.fit_transform(test[['transmission','fuelType']])
ohe_df_test = pd.DataFrame(ohe_test, columns=ohe.get_feature_names_out())
test_df = pd.concat([ohe_df_test, test.drop(['transmission','fuelType'], axis=1)],axis=1)

print(train_df)
print(test_df)

x = train_df.drop(['price'],axis=1)
y = train_df['price']

      transmission_Automatic  transmission_Manual  transmission_Semi-Auto  \
0                        1.0                  0.0                     0.0   
1                        1.0                  0.0                     0.0   
2                        1.0                  0.0                     0.0   
3                        1.0                  0.0                     0.0   
4                        1.0                  0.0                     0.0   
...                      ...                  ...                     ...   
6894                     0.0                  0.0                     1.0   
6895                     0.0                  1.0                     0.0   
6896                     0.0                  0.0                     1.0   
6897                     0.0                  0.0                     1.0   
6898                     0.0                  0.0                     1.0   

      fuelType_Diesel  fuelType_Hybrid  fuelType_Other  fuelType_Petrol  \


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import numpy as np

x_train, x_valid, y_train, y_valid = train_test_split(x,y,test_size=0.2)

model = RandomForestRegressor(n_estimators=100, max_depth=20)
model.fit(x_train, y_train)

pred = model.predict(x_valid)
print(pred)
print(mean_squared_error(y_valid, pred, squared=False))
print(np.sqrt(mean_squared_error(y_valid, pred, squared=False)))

[45096.68833333 13137.57       23632.48       ... 18563.14533333
 20037.38066987 32720.49833333]
2548.4080331036685
50.48175940974788


In [ ]:
from sklearn.model_selection import GridSearchCV

params = {'n_estimators': [5,10,30,50,100], 'max_depth':[2,5,10,20,30]}
model = RandomForestRegressor()
gscv = GridSearchCV(estimator = model, param_grid = params, cv=3)
gscv.fit(x_train,y_train)
print(gscv.best_params_)

{'max_depth': 20, 'n_estimators': 100}


In [ ]:
x_test = test_df

x_test['fuelType_Other'] = x_test['fuelType_Other'] + x_test['fuelType_Electric']
x_test = x_test.drop(['fuelType_Electric'], axis=1)

pred = model.predict(x_test)

result = pd.DataFrame({'pred': pred})
result.to_csv("수험번호", index=False)
print(result)

              pred
0     25542.240000
1     32362.151307
2     30781.292370
3     25470.180000
4     39134.720000
...            ...
3877  37668.300000
3878  28014.753333
3879  26083.345833
3880  20950.750000
3881  26144.160000

[3882 rows x 1 columns]
